# BioSimSpace diaries: Alchemical Relative Free Energy Calculations

Alchemical Free Energy (AFE) calculations have become an important technique in computer-aided drug design. If you are relatively new to this field you may have come across the acronym FEP in the context of protein-ligand binding affinity estimations. FEP stands for Free Energy Perturbation, which is just one way to implement an AFE calculation. For more methodological background oreading the best-practices LiveCoMs article from [Mey et al.](https://livecomsjournal.org/index.php/livecoms/article/view/v2i1e18378) is recommended, whereas the review JCIM article from [Cournia et al.](https://pubs.acs.org/doi/full/10.1021/acs.jcim.7b00564) gives a good modern perspective on the status of AFE calculations in drug discovery. 

AFE calculations have a reputation for having a steep-learning curve. There's a lot of that can go wrong when setting up or analysing AFE calculations. [BioSimSpace](www.biosimspace.openbiosim.org) has made it easier for non-experts to integrate AFE calculations in python-based drug design workflows. 

This post will give an overview of the AFE functionality available in BioSimSpace 2023.3. We will focus today on relative free energy calculations. 


## A simple example first

We start with the venerable problem of computing the relative hydration free energy of ethane to methanol, first reported by [Jorgensen and Ravimohan](https://pubs.aip.org/aip/jcp/article-abstract/83/6/3050/89020/Monte-Carlo-simulation-of-differences-in-free?redirectedFrom=fulltext) in 1985. Here we are interested in calculating the difference in free energy changes upon hydrating a molecule of methanol and a molecule of ethane. 

![A thermodynamic cycle for the calculation of the relative hydration free energy of ethane to methanol](e2m.png)

Because free energy is a state function, the total free energy change around the cycle is 0:

$\Delta G^{\mathrm{ethane}}_{\mathrm{hyd}} + \Delta G_{\mathrm{solv}} - \Delta G^{\mathrm{methanol}}_{\mathrm{hyd}} -  \Delta G_{\mathrm{vac}} = 0$

This allows us to obtain $\Delta \Delta G_{\mathrm{hyd,\: ethane-methanol}}$ in terms of $\Delta G_{\mathrm{solv}}$ and $\Delta G_{\mathrm{vac}}$:

$\Delta G^{\mathrm{methanol}}_{\mathrm{hyd}}  - \Delta G^{\mathrm{ethane}}_{\mathrm{hyd}} = \Delta G_{\mathrm{solv}} - \Delta G_{\mathrm{vac}}$

$\Delta \Delta G_{\mathrm{hyd,\: ethane-methanol}} = \Delta G_{\mathrm{solv}} - \Delta G_{\mathrm{vac}}$

Now we just need to compute these quantities using alchemical simulations.

Let's load first input files

In [1]:
import BioSimSpace as BSS

INFO:rdkit:Enabling RDKit 2023.03.2 jupyter extensions
/Users/julien/mambaforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/IO/_io.py:410: UserWarning: BioSimSpace.IO: Please install GROMACS (http://www.gromacs.org) for GROMACS topology file support.
  _warnings.warn(
/Users/julien/mambaforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/Process/_somd.py:407: UserWarning: No simulation box found. Assuming gas phase simulation.
  _warnings.warn("No simulation box found. Assuming gas phase simulation.")
/Users/julien/mambaforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/_Config/_config.py:96: UserWarning: No simulation box found. Assuming gas phase simulation.
  _warnings.warn("No simulation box found. Assuming gas phase simulation.")
/Users/julien/mambaforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/_Config/_somd.py:209: UserWarning: Cannot use a barostat for a vacuum or non-periodic simulation
  _warnings.warn(


In [2]:
# We assume the molecules to perturb are the first molecules in each system. (Each file contains a single molecule.)
# we use [0] to select this first molecule.
ethane = BSS.IO.readMolecules("./ethane.pdb")[0]
methanol = BSS.IO.readMolecules("./methanol.pdb")[0]

In [4]:
BSS.Notebook.View(ethane).system()

ThemeManager()

NGLWidget(gui_style='ngl')

In [5]:
BSS.Notebook.View(methanol).system()

NGLWidget(gui_style='ngl')

In [6]:
ethane = BSS.Parameters.gaff(ethane).getMolecule()
methanol = BSS.Parameters.gaff(methanol).getMolecule()

In [8]:
mapping = BSS.Align.matchAtoms(ethane, methanol)
print (mapping)

{0: 0, 1: 2, 2: 1, 3: 4, 4: 3, 6: 5}


In [9]:
BSS.Align.viewMapping(ethane, methanol, mapping)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
inv_mapping = {v: k for k, v in mapping.items()}
methanol_aligned = BSS.Align.rmsdAlign(methanol, ethane, inv_mapping)

In [14]:
merged = BSS.Align.merge(ethane, methanol_aligned, mapping)

In [15]:
protocol = BSS.Protocol.FreeEnergy(runtime=0.4*BSS.Units.Time.nanosecond, 
                                   report_interval=10000,
                                   restart_interval=100, num_lam=3)

In [16]:
fep_vac = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, work_dir="ethane_methanol_somd/vacuum")

In [17]:
fep_vac.run() # fep_vac.wait()

In [18]:
pmf_vac, overlap_matrix_vac = BSS.FreeEnergy.Relative.analyse(f'ethane_methanol_somd/vacuum')

In [25]:
print ("The free energy change is DG_vac = %s +/- %s"% (pmf_vac[-1][1], pmf_vac[-1][2]))

The free energy change is DG_vac = 2.4297 kcal/mol +/- 0.0582 kcal/mol


In [26]:
fep_vac_gmx = BSS.FreeEnergy.Relative(merged.toSystem(), protocol, 
                                      engine="gromacs",
                                      work_dir="ethane_methanol_gmx/vacuum")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 fep_vac_gmx = BSS.FreeEnergy.Relative(merged.toSystem(), protocol,                           │
│   2 │   │   │   │   │   │   │   │   │     engine="gromacs",                                      │
│   3 │   │   │   │   │   │   │   │   │     work_dir="ethane_methanol_gmx/vacuum")                 │
│   4                                                                                              │
│                                                                                                  │
│ /Users/julien/mambaforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/FreeEnergy/_rel │
│ ative.py:185 in __init__                                                                         │
│                                                                                                  │
│    182 │   │   │   # Make sure GROMACS is installed if GROMACS engine is selected.               │
│    183 │   │   │   if engine == "GROMACS":                                                       │
│    184 │   │   │   │   if _gmx_exe is None:                                                      │
│ ❱  185 │   │   │   │   │   raise _MissingSoftwareError(                                          │
│    186 │   │   │   │   │   │   "Cannot use GROMACS engine as GROMACS is not installed!"          │
│    187 │   │   │   │   │   )                                                                     │
│    188                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
MissingSoftwareError: Cannot use GROMACS engine as GROMACS is not installed!